In [1]:
import findspark
findspark.init()
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import Window
from pyspark.sql.functions import *
import pandas as pd
import os

spark = SparkSession.builder.master('local[1]').appName('bbi').getOrCreate()

24/01/27 17:26:28 WARN Utils: Your hostname, Tech-Buddy resolves to a loopback address: 127.0.1.1; using 172.17.35.137 instead (on interface eth0)
24/01/27 17:26:28 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/01/27 17:26:29 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
data = [['Scott', 50], ['Jeff', 45], ['Thomas', 54],['Ann',34]] 
 
# Create the pandas DataFrame 
pandasDF = pd.DataFrame(data, columns = ['Name', 'Age']) 
json_data = pandasDF.to_json(orient='records')
pandas_df = pd.read_json(json_data)
# print dataframe. 
print(pandasDF)

     Name  Age
0   Scott   50
1    Jeff   45
2  Thomas   54
3     Ann   34


/tmp/ipykernel_1102/3740610486.py:6: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  pandas_df = pd.read_json(json_data)


In [4]:
# Create the pandas DataFrame 
pandasDF = pd.DataFrame(data, columns = ['Name', 'Age']) 
json_data = pandasDF.to_json(orient='records')
pandas_df = pd.read_json(json_data)
# print dataframe. 
print(pandasDF)
sparkDF=spark.createDataFrame(pandas_df) 
sparkDF.printSchema()
sparkDF.show()

/tmp/ipykernel_1102/2452353148.py:4: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  pandas_df = pd.read_json(json_data)


     Name  Age
0   Scott   50
1    Jeff   45
2  Thomas   54
3     Ann   34
root
 |-- Name: string (nullable = true)
 |-- Age: long (nullable = true)



+------+---+
|  Name|Age|
+------+---+
| Scott| 50|
|  Jeff| 45|
|Thomas| 54|
|   Ann| 34|
+------+---+



In [5]:
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

# Create a Spark session
spark = SparkSession.builder.appName("example").getOrCreate()

# Sample data
data = [['Scott', 50], ['Jeff', 45], ['Thomas', 54], ['Ann', 34]]

# Create a Pandas DataFrame
pandasDF = pd.DataFrame(data, columns=['Name', 'Age'])

# Convert Pandas DataFrame to JSON
json_data = pandasDF.to_json(orient='records')

# Convert JSON string to Pandas DataFrame
pandas_df = pd.read_json(json_data)

# Create PySpark DataFrame from Pandas DataFrame
schema = StructType([
    StructField("Name", StringType(), True),
    StructField("Age", IntegerType(), True)
])
sparkDF = spark.createDataFrame(pandas_df, schema=schema)

# Show PySpark DataFrame
sparkDF.printSchema()
sparkDF.show()

24/01/22 20:25:34 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.
/tmp/ipykernel_1102/4255211086.py:18: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  pandas_df = pd.read_json(json_data)


root
 |-- Name: string (nullable = true)
 |-- Age: integer (nullable = true)

+------+---+
|  Name|Age|
+------+---+
| Scott| 50|
|  Jeff| 45|
|Thomas| 54|
|   Ann| 34|
+------+---+



In [6]:
li = [1, 2, 3, 1, 4, 2, 8, 9]
rdd = spark.sparkContext.parallelize(li)
rdd.collect()

[1, 2, 3, 1, 4, 2, 8, 9]

In [7]:
rdd1 = rdd.distinct()
rdd1.collect()

[1, 2, 3, 4, 8, 9]

In [20]:
from pyspark.sql import SparkSession

# Create a Spark session
spark = SparkSession.builder.appName("example").getOrCreate()

# Your list
li = [1, 2, 3, 1, 4, 2, 8, 9]

# Create an RDD from the list
rdd = spark.sparkContext.parallelize(li)

# Identify and replace duplicates with 'x'
result_rdd = rdd.map(lambda x: (x, 1)).reduceByKey(lambda x, y: x + y).flatMap(lambda x: [x[0]] + ['x'] * (x[1] - 1))

# Show the original and updated RDDs
print("Original RDD:", rdd.collect())
print("Updated RDD:", result_rdd.collect())


Original RDD: [1, 2, 3, 1, 4, 2, 8, 9]
Updated RDD: [1, 'x', 2, 'x', 3, 4, 8, 9]


In [32]:
from pyspark.sql import SparkSession

# Create a Spark session
spark = SparkSession.builder.appName("example").getOrCreate()

# Your list
li = [1, 2, 3, 1, 4, 2, 8, 9]

# Create an RDD from the list
rdd = spark.sparkContext.parallelize(li)

# Identify and replace duplicates (except the first occurrence) with 'x'
result_rdd = (
    rdd.zipWithIndex()
    .map(lambda x: (x[0], x[1], li.index(x[0])))
    .map(lambda x: (x[0], 'x') if x[2] != x[1] else (x[0], x[0]))
)

# Show the original and updated RDDs
print("Original RDD:", rdd.collect())
print("Updated RDD:", result_rdd.values().collect())
rdd.zipWithIndex().map(lambda x: (x[0], x[1], li.index(x[0]))).collect()

Original RDD: [1, 2, 3, 1, 4, 2, 8, 9]
Updated RDD: [1, 2, 3, 'x', 4, 'x', 8, 9]


[(1, 0, 0),
 (2, 1, 1),
 (3, 2, 2),
 (1, 3, 0),
 (4, 4, 4),
 (2, 5, 1),
 (8, 6, 6),
 (9, 7, 7)]

In [ ]:
li = [1, 2, 3, 1, 4, 2, 8, 9]

# Create an RDD from the list
rdd = spark.sparkContext.parallelize(li)
rdd1 = rdd.zipwithindex().map(

In [34]:
from pyspark.sql import SparkSession

# Create a Spark session
spark = SparkSession.builder.appName("example").getOrCreate()

# Your list
li = [1, 2, 3, 1, 4, 2, 8, 9]

# Create an RDD from the list
rdd = spark.sparkContext.parallelize(li)

# Identify and replace duplicates (except the first occurrence) with 'x'
result_rdd = (
    rdd.zipWithIndex()
    .map(lambda x: (x[1], x[0], li.index(x[0])))
    .flatMap(lambda x: [('x',)] * (x[2] != x[1]) + [(x[0],)])
)

# Show the original and updated RDDs
print("Original RDD:", rdd.collect())
print("Updated RDD:", result_rdd.values().collect())


Original RDD: [1, 2, 3, 1, 4, 2, 8, 9]


24/01/15 12:58:48 ERROR Executor: Exception in task 0.0 in stage 91.0 (TID 98)
org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/home/basu/spark/python/lib/pyspark.zip/pyspark/worker.py", line 1247, in main
    process()
  File "/home/basu/spark/python/lib/pyspark.zip/pyspark/worker.py", line 1239, in process
    serializer.dump_stream(out_iter, outfile)
  File "/home/basu/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 274, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/home/basu/spark/python/lib/pyspark.zip/pyspark/util.py", line 83, in wrapper
    return f(*args, **kwargs)
  File "/home/basu/spark/python/pyspark/rdd.py", line 3503, in <lambda>
    return self.map(lambda x: x[1])
IndexError: tuple index out of range

	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.handlePythonException(PythonRunner.scala:572)
	at org.apache.spark.api.python.PythonRunner$$anon$3.read(PythonRunner.scala:784)
	at

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 91.0 failed 1 times, most recent failure: Lost task 0.0 in stage 91.0 (TID 98) (172.17.35.137 executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/home/basu/spark/python/lib/pyspark.zip/pyspark/worker.py", line 1247, in main
    process()
  File "/home/basu/spark/python/lib/pyspark.zip/pyspark/worker.py", line 1239, in process
    serializer.dump_stream(out_iter, outfile)
  File "/home/basu/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 274, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/home/basu/spark/python/lib/pyspark.zip/pyspark/util.py", line 83, in wrapper
    return f(*args, **kwargs)
  File "/home/basu/spark/python/pyspark/rdd.py", line 3503, in <lambda>
    return self.map(lambda x: x[1])
IndexError:

In [60]:
from pyspark.sql import SparkSession

# Create a Spark session
spark = SparkSession.builder.appName("example").getOrCreate()

# Create an RDD from a text file
rdd = spark.sparkContext.textFile('/home/basu/data/x.txt')
rdd1 = rdd.flatMap(lambda x: x.split(' '))

# Function to process each partition and replace duplicates
def process_partition(index, iterator):
    unique_elements_in_partition = set()
    for x in iterator:
        if x not in unique_elements_in_partition:
            unique_elements_in_partition.add(x)
            yield x
        else:
            yield 'x'

# Use mapPartitionsWithIndex to apply the function to each partition
result_rdd = rdd1.mapPartitionsWithIndex(process_partition)

# Show the original and updated RDDs
print("Original RDD:", rdd1.collect())
print("Updated RDD:", result_rdd.collect())


Original RDD: ['We', 'can', 'rollback', 'a', 'delete', 'query', 'but', 'not', 'so', 'for', 'truncate', 'and', 'drop.', 'When', 'I', 'execute', 'queries', 'then', 'successfully', 'done', 'with', 'rollback', 'in', 'delete,', 'drop', '&', 'truncate.', '', 'We', 'can', 'rollback', 'the', 'data', 'in', 'conditions', 'of', 'Delete,', 'Truncate', '&', 'Drop.', 'But', 'must', 'be', 'used', 'Begin', 'Transaction', 'before', 'executing', 'query', 'Delete,', 'Drop', '&', 'Truncate.']
Updated RDD: ['We', 'can', 'rollback', 'a', 'delete', 'query', 'but', 'not', 'so', 'for', 'truncate', 'and', 'drop.', 'When', 'I', 'execute', 'queries', 'then', 'successfully', 'done', 'with', 'x', 'in', 'delete,', 'drop', '&', 'truncate.', '', 'x', 'x', 'x', 'the', 'data', 'x', 'conditions', 'of', 'Delete,', 'Truncate', 'x', 'Drop.', 'But', 'must', 'be', 'used', 'Begin', 'Transaction', 'before', 'executing', 'x', 'x', 'Drop', 'x', 'Truncate.']


In [10]:
def example_function(arg1, *args, kwarg1="default_value", **kwargs):
    print("arg1:", arg1)
    print("args:", args)
    print("kwarg1:", kwarg1)
    print("kwargs:", kwargs)

# Calling the function with various arguments
example_function(1, 2, 3, kwarg1="custom_value", key1="value1", key2="value2")


arg1: 1
args: (2, 3)
kwarg1: custom_value
kwargs: {'key1': 'value1', 'key2': 'value2'}
